In [ ]:
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [ ]:
# create a list of strings (the strings have to be sentences that will be the most commonly used in daily lives)
text_daa

In [ ]:
# prompt: Generate a list of strings which are the most commonly occuring sentences in daily lives

text_data = [
    "I'm going to the store to buy some milk.",
    "Can you pass me the remote control?",
    "I'm feeling a bit tired today.",
    "The weather forecast says it will rain tomorrow.",
    "I need to finish this project by Friday.",
    "The new restaurant downtown has great reviews.",
    "I'm not sure what to wear to the party.",
    "Can you help me carry these heavy boxes?",
    "I'm going to the gym to work out.",
    "The movie starts at 7 PM tonight.",
    "I need to call my mom to check in.",
    "The traffic on the highway is terrible today.",
    "I'm going to the bank to deposit my paycheck.",
    "The new smartphone model is really expensive.",
    "I'm not feeling well, so I'm going to stay home.",
    "The concert tickets are selling out fast.",
    "I need to pick up my dry cleaning on the way home.",
    "The new book by my favorite author is out now.",
    "I'm going to the park to play some soccer.",
    "The new restaurant has a great selection of vegetarian options."
]

In [ ]:
# prepare the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [ ]:
total_words

101

In [ ]:
tokenizer.word_index

{'the': 1,
 'to': 2,
 "i'm": 3,
 'going': 4,
 'new': 5,
 'my': 6,
 'i': 7,
 'need': 8,
 'out': 9,
 'is': 10,
 'some': 11,
 'can': 12,
 'you': 13,
 'me': 14,
 'feeling': 15,
 'a': 16,
 'today': 17,
 'by': 18,
 'restaurant': 19,
 'has': 20,
 'great': 21,
 'not': 22,
 'on': 23,
 'home': 24,
 'store': 25,
 'buy': 26,
 'milk': 27,
 'pass': 28,
 'remote': 29,
 'control': 30,
 'bit': 31,
 'tired': 32,
 'weather': 33,
 'forecast': 34,
 'says': 35,
 'it': 36,
 'will': 37,
 'rain': 38,
 'tomorrow': 39,
 'finish': 40,
 'this': 41,
 'project': 42,
 'friday': 43,
 'downtown': 44,
 'reviews': 45,
 'sure': 46,
 'what': 47,
 'wear': 48,
 'party': 49,
 'help': 50,
 'carry': 51,
 'these': 52,
 'heavy': 53,
 'boxes': 54,
 'gym': 55,
 'work': 56,
 'movie': 57,
 'starts': 58,
 'at': 59,
 '7': 60,
 'pm': 61,
 'tonight': 62,
 'call': 63,
 'mom': 64,
 'check': 65,
 'in': 66,
 'traffic': 67,
 'highway': 68,
 'terrible': 69,
 'bank': 70,
 'deposit': 71,
 'paycheck': 72,
 'smartphone': 73,
 'model': 74,
 'really

Prepare the data

In [ ]:
#create sequences of words
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

Padding of sequences

In [ ]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

In [ ]:
input_sequences

array([[  0,   0,   0, ...,   0,   3,   4],
       [  0,   0,   0, ...,   3,   4,   2],
       [  0,   0,   0, ...,   4,   2,   1],
       ...,
       [  0,   0,   0, ...,  21,  97,  98],
       [  0,   0,   0, ...,  97,  98,  99],
       [  0,   0,   1, ...,  98,  99, 100]], dtype=int32)

In [ ]:
## split the data into features and labels
input_sequences = np.array(input_sequences)
X,y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
input_sequences[:,-1]

array([  4,   2,   1,  25,   2,  26,  11,  27,  13,  28,  14,   1,  29,
        30,  15,  16,  31,  32,  17,  33,  34,  35,  36,  37,  38,  39,
         8,   2,  40,  41,  42,  18,  43,   5,  19,  44,  20,  21,  45,
        22,  46,  47,   2,  48,   2,   1,  49,  13,  50,  14,  51,  52,
        53,  54,   4,   2,   1,  55,   2,  56,   9,  57,  58,  59,  60,
        61,  62,   8,   2,  63,   6,  64,   2,  65,  66,  67,  23,   1,
        68,  10,  69,  17,   4,   2,   1,  70,   2,  71,   6,  72,   5,
        73,  74,  10,  75,  76,  22,  15,  77,  78,   3,   4,   2,  79,
        24,  80,  81,  82,  83,   9,  84,   8,   2,  85,  86,   6,  87,
        88,  23,   1,  89,  24,   5,  90,  18,   6,  91,  92,  10,   9,
        93,   4,   2,   1,  94,   2,  95,  11,  96,   5,  19,  20,  16,
        21,  97,  98,  99, 100], dtype=int32)

In [ ]:
y[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [ ]:
# Build the RNN model
model = Sequential()
model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 10)            1010      
                                                                 
 lstm (LSTM)                 (None, 11, 150)           96600     
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 101)               10201     
                                                                 
Total params: 208211 (813.32 KB)
Trainable params: 208211 (813.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# fit the model
model.fit(X, y, epochs=200, batch_size=1)

Epoch 1/200
148/148 [==============================] - 8s 27ms/step - loss: 4.6216 - accuracy: 0.0338
Epoch 2/200
148/148 [==============================] - 3s 18ms/step - loss: 4.5541 - accuracy: 0.0946
Epoch 3/200
148/148 [==============================] - 3s 18ms/step - loss: 4.3770 - accuracy: 0.1014
Epoch 4/200
148/148 [==============================] - 3s 21ms/step - loss: 4.2436 - accuracy: 0.1081
Epoch 5/200
148/148 [==============================] - 5s 31ms/step - loss: 4.0897 - accuracy: 0.1081
Epoch 6/200
148/148 [==============================] - 3s 19ms/step - loss: 3.9153 - accuracy: 0.1081
Epoch 7/200
148/148 [==============================] - 3s 18ms/step - loss: 3.7806 - accuracy: 0.1149
Epoch 8/200
148/148 [==============================] - 3s 19ms/step - loss: 3.6050 - accuracy: 0.1284
Epoch 9/200
148/148 [==============================] - 4s 26ms/step - loss: 3.4093 - accuracy: 0.1554
Epoch 10/200
148/148 [==============================] - 4s 24ms/step - loss: 3.174

In [ ]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list)
    predicted_index = np.argmax(predicted)
    predicted_word = tokenizer.index_word[predicted_index]
    return predicted_word

new  = "I'm"

next_word = predict_next_word(model, tokenizer, new, max_sequence_len)
print(f"Next Word: {next_word}")

1/1 [==============================] - 0s 28ms/step
Next Word: going
